In [282]:
import pandas as pd
import random
import os
import numpy as np
import re

directory1 = '/home/rodrigo/Área de Trabalho/PT2_CARBU/VENDAS_INTECH'
files1 = os.listdir(directory1)

directory2 = '/home/rodrigo/Área de Trabalho/PT2_CARBU/ParcelasClientes'
files2 = os.listdir(directory2)

def read_file(file_path) -> list[list[str]]:
    line_list = list()
    with open(file_path, 'r', encoding='latin-1') as file:
        lines =  file.readlines()
        [line_list.append(line.strip().split()) for line in lines if not line.isspace()]
    
    return line_list

def getId() -> int:
    return int(data1[5][1])

def getFinanciado() -> bool:
    return not data1[4][-1] == 'VISTA'

def searchFile() -> str:
    for file in files2:
        id = getId()
        
        pattern = r'\d+'
        matche = re.findall(pattern, file)

        idClient = [int(match) for match in matche]

        if id == idClient[0]:
            return file

    return ''

def getDataAqui() -> list:
        for word in data1[3]:
            pattern = r'^\d{2}/\d{2}/\d{4}$'
        if re.match(pattern, word):
            return [word]
        return ['']


def getValorAqui() -> list:

    for line in data1[1:]:
        if len(line) > 1 and line[1] == "VENDA:":
            return [line[-1]]
        
def getValorParcelas() -> float:
    if not getFinanciado():
        return [0.0]

    line = 20
    try:
        valor = data1[line][-1].replace(',', '.')
        return [float(valor)]
    except:
        return [0.0]
    
def getQtdFinanciamento() -> int:
    try:
        numero = int(''.join(filter(str.isdigit, data1[4][-1])))
        return numero
    except:
        return 0
    
def getValorNominal() -> list:
    i = getQtdFinanciamento()
    if i < 0:
        raise ValueError("O valor de i deve ser não negativo.")

    vetor = []

    for k in range(i + 1):
        termo = int(getValorParcelas()[0]) / (1 + 0.04) ** k
        vetor.append(round(termo, 2))

    return vetor[1:]

def getNome() -> str:
    nome = list()
    stop_re = r'^[0-9]+$'
    for i in reversed(data1[5]):
        nome.insert(0, i)
        if re.match(stop_re, i):
            break
    
    return [' '.join(nome[2:-2])]

def getP() -> str:
    counter: int = 0
    for i, line in enumerate(data1):
        if len(line) == 1:
            counter += 1
        if counter == 3:
            tmp = data1[i-1]
            break

    return [" ".join(tmp[2:5])]

def getDataVencimento() -> str:

        regex_data = r'\b\d{2}/\d{2}/\d{4}\b' 
        datas = []

        for linha in data2[10:]:
            if len(linha) > 2 and not (linha[0].isdigit()):
                if re.match(regex_data, linha[2]):
                    datas.insert(0, linha[2])
                elif re.match(regex_data, linha[1]):
                    datas.insert(0, linha[1])
            elif len(linha) > 2 and (linha[0].isdigit()) and re.match(regex_data, linha[2]):
                datas.insert(0, linha[2])

        return datas[1:]
    
def getValorRecebido() -> list[float]:
    values = list()
    regex_value = r'\b\d{1,5},\d{2}\b'

    for line in data2[11:]:
        for i in range(len(line)):
            if re.match(regex_value, line[i]):
                values.insert(0, line[i+1])
                break
        if len(line) <= 1:
            return values[1:]

for file1 in files1:
    file_path1= os.path.join(directory1, file1)
    data1 = read_file(file_path1)

    file2 = searchFile()
    if file2 != '':    
      file_path2= os.path.join(directory2, file2)
      data2 = read_file(file_path2)
    else:
       continue
            
    dataaquisicao = getDataAqui()
    valoraqui = getValorAqui()
    datavencimento = getDataVencimento()
    valornominal = getValorNominal()
    valorrecebido = getValorRecebido()
    nome = getNome()
    produto = getP()

    tamanhos = [len(dataaquisicao), len(valoraqui), len(datavencimento), len(valornominal), len(valorrecebido), len(nome), len(produto)]

    tamanho_maximo = max(tamanhos)

    dataaquisicao += [None] * (tamanho_maximo - len(dataaquisicao))
    valoraqui += [None] * (tamanho_maximo - len(valoraqui))
    datavencimento += [None] * (tamanho_maximo - len(datavencimento))
    valornominal += [None] * (tamanho_maximo - len(valornominal))
    valorrecebido += [None] * (tamanho_maximo - len(valorrecebido))
    nome += [None] * (tamanho_maximo - len(nome))
    produto += [None] * (tamanho_maximo - len(produto))

    colunas_para_preencher = {
        'Data da Aquisição': dataaquisicao,
        'Valor de Aquisição': valoraqui,
        'Data de Vencimento': datavencimento,
        'Valor Nominal': valornominal,
        'Data do Pagamento Recebido': [None] * tamanho_maximo,
        'Valor do Pagamento Recebido': valorrecebido,
        'Tipo de Ativo': [None] * tamanho_maximo,
        'Cedente': [None] * tamanho_maximo,
        'Sacado': nome,
        'Produto': produto,
    }

    try:
        novos_dados_df = pd.DataFrame(colunas_para_preencher)
        df = pd.concat([df, novos_dados_df], ignore_index=True)
        df.to_excel('novatabela.xlsx', index=False)
        print(df)
    except ValueError as e:
        print(f"Ocorreu um erro: {e}")

df

/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)


   Data da Aquisição Valor de Aquisição Data de Vencimento  Valor Nominal  \
0                              5.800,00         16/11/2018         259.62   
1               None               None         16/12/2018         249.63   
2               None               None         16/01/2019         240.03   
3               None               None         16/04/2019         230.80   
4               None               None         16/05/2019         221.92   
5               None               None         16/06/2019         213.38   
6               None               None         16/07/2019         205.18   
7               None               None         16/08/2019         197.29   
8               None               None         16/09/2019         189.70   
9               None               None         16/12/2019         182.40   
10              None               None         16/01/2020         175.39   
11              None               None         16/02/2020         168.64   

/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)
/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)


    Data da Aquisição Valor de Aquisição Data de Vencimento  Valor Nominal  \
0                               5.800,00         16/11/2018         259.62   
1                None               None         16/12/2018         249.63   
2                None               None         16/01/2019         240.03   
3                None               None         16/04/2019         230.80   
4                None               None         16/05/2019         221.92   
..                ...                ...                ...            ...   
387              None               None         12/12/2017            NaN   
388              None               None         12/01/2018            NaN   
389              None               None         12/02/2018            NaN   
390              None               None         12/03/2018            NaN   
391              None               None         12/04/2018            NaN   

    Data do Pagamento Recebido Valor do Pagamento Recebido Tipo

/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)
/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)


...

/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)


     Data da Aquisição Valor de Aquisição Data de Vencimento  Valor Nominal  \
0                                5.800,00         16/11/2018         259.62   
1                 None               None         16/12/2018         249.63   
2                 None               None         16/01/2019         240.03   
3                 None               None         16/04/2019         230.80   
4                 None               None         16/05/2019         221.92   
...                ...                ...                ...            ...   
2685              None               None         28/04/2025          95.93   
2686              None               None         28/05/2025          92.24   
2687              None               None               None          88.70   
2688                            10.400,00         26/05/2020            NaN   
2689              None               None         01/06/2020            NaN   

     Data do Pagamento Recebido Valor do Pagamento 

/tmp/ipykernel_40199/3089532817.py:181: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, novos_dados_df], ignore_index=True)


     Data da Aquisição Valor de Aquisição Data de Vencimento  Valor Nominal  \
0                                5.800,00         16/11/2018         259.62   
1                 None               None         16/12/2018         249.63   
2                 None               None         16/01/2019         240.03   
3                 None               None         16/04/2019         230.80   
4                 None               None         16/05/2019         221.92   
...                ...                ...                ...            ...   
2843              None               None         10/06/2019         161.05   
2844              None               None         10/07/2019         154.86   
2845              None               None         10/08/2019         148.90   
2846              None               None         10/09/2019         143.17   
2847                             5.000,00         25/05/2020            NaN   

     Data do Pagamento Recebido Valor do Pagamento 

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import random
import os
import numpy as np
import re

In [ ]:
directory1 = '/home/rodrigo/Área de Trabalho/PT2_CARBU/VENDAS_INTECH'
files1 = os.listdir(directory1)

def read_file1(file_path) -> list[list[str]]:
    line_list = list()
    with open(file_path, 'r', encoding='latin-1') as file:
        lines =  file.readlines()
        [line_list.append(line.strip().split()) for line in lines if not line.isspace()]
    
    return line_list

file1 = random.choice(files1)
file_path1 = os.path.join(directory1, file1)
data1 = read_file1(file_path1)
for i, line in enumerate(data1):
    print(i, line)

In [ ]:
def getId() -> int:
    return int(data1[5][1])

getId()

In [ ]:
def getFinanciado() -> bool:
    return not data1[4][-1] == 'VISTA'

getFinanciado()

In [ ]:
directory2 = '/home/rodrigo/Área de Trabalho/PT2_CARBU/ParcelasClientes'
files2 = os.listdir(directory2)

In [ ]:
def searchFile() -> str:
  for file in files2:
    id = getId()
    
    pattern = r'\d+'
    matche = re.findall(pattern, file)

    idClient = [int(match) for match in matche]

    if id == idClient[0]:
      return file

  return ''

searchFile()

In [ ]:
def read_file(file_path) -> list[list[str]]:
    line_list = list()
    with open(file_path, 'r', encoding='latin-1') as file:
        lines =  file.readlines()
        [line_list.append(line.strip().split()) for line in lines if not line.isspace()]
    
    return line_list

try:
    file = searchFile()
    file_path = os.path.join(directory2, file)
    data2 = read_file(file_path)
    for i, line in enumerate(data2):
        print(i, line)
except:
    print("Cliente não é ativo")

In [ ]:
def getDataAqui() -> list:
  for word in data1[3]:
    pattern = r'^\d{2}/\d{2}/\d{4}$'
    if re.match(pattern, word):
      return [word]
  return ['']

getDataAqui()

In [ ]:
def getValorAqui() -> list:

    for line in data1[1:]:
        if len(line) > 1 and line[1] == "VENDA:":
            return [line[-1]]

getValorAqui()

In [ ]:
def getDataVencimento() -> str:

    regex_data = r'\b\d{2}/\d{2}/\d{4}\b' 
    datas = []

    for linha in data2[10:]:
        if len(linha) > 2 and not (linha[0].isdigit()):
            if re.match(regex_data, linha[2]):
                datas.insert(0, linha[2])
            elif re.match(regex_data, linha[1]):
                datas.insert(0, linha[1])
        elif len(linha) > 2 and (linha[0].isdigit()) and re.match(regex_data, linha[2]):
            datas.insert(0, linha[2])

    return datas[1:]

getDataVencimento()

In [ ]:
def getValorParcelas() -> float:
    if not getFinanciado():
        return [0.0]

    line = 20
    try:
        valor = data1[line][-1].replace(',', '.')
        return [float(valor)]
    except:
        return [0.0]


getValorParcelas()

In [ ]:
def getQtdFinanciamento() -> int:
    try:
        numero = int(''.join(filter(str.isdigit, data1[4][-1])))
        return numero
    except:
        return 0


def getValorNominal() -> list:
  i = getQtdFinanciamento()
  if i < 0:
    raise ValueError("O valor de i deve ser não negativo.")

  vetor = []

  for k in range(i + 1):
    termo = int(getValorParcelas()[0]) / (1 + 0.04) ** k
    vetor.append(round(termo, 2))

  return vetor[1:]

getValorNominal()

In [ ]:
def getValorRecebido() -> list[float]:
  values = list()
  regex_value = r'\b\d{1,5},\d{2}\b'

  for line in data2[11:]:
    for i in range(len(line)):
      if re.match(regex_value, line[i]):
        values.insert(0, line[i+1])
        break
    if len(line) <= 1:
      return values[1:]

getValorRecebido()

In [ ]:
def getNome() -> str:
  nome = list()
  stop_re = r'^[0-9]+$'
  for i in reversed(data1[5]):
    nome.insert(0, i)
    if re.match(stop_re, i):
      break
  
  return [' '.join(nome[2:-2])]

getNome()

In [ ]:
def getP() -> str:
  counter: int = 0
  for i, line in enumerate(data1):
    if len(line) == 1:
      counter += 1
    if counter == 3:
      tmp = data1[i-1]
      break

  return [" ".join(tmp[2:5])]

getP()

# PREENCHIMENTO

In [275]:
df = pd.read_excel('novatabela.xlsx')

df

,Data da Aquisição,Valor de Aquisição,Data de Vencimento,Valor Nominal,Data do Pagamento Recebido,Valor do Pagamento Recebido,Tipo de Ativo,Cedente,Sacado,Produto
0,03/05/2019,"4.900,00",03/06/2019,228.85,NaN,"232,00",NaN,NaN,IVANILDO LORENTINO,POP 100 2014
1,NaN,NaN,03/07/2019,220.04,NaN,"238,00",NaN,NaN,NaN,NaN
2,NaN,NaN,03/08/2019,211.58,NaN,"238,00",NaN,NaN,NaN,NaN
3,NaN,NaN,03/09/2019,203.44,NaN,"236,50",NaN,NaN,NaN,NaN
4,NaN,NaN,03/10/2019,195.62,NaN,"236,50",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
521,NaN,NaN,NaN,108.62,NaN,NaN,NaN,NaN,NaN,NaN
522,NaN,NaN,NaN,104.44,NaN,NaN,NaN,NaN,NaN,NaN
523,NaN,NaN,NaN,100.43,NaN,NaN,NaN,NaN,NaN,NaN
524,NaN,NaN,NaN,96.56,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tamanho_maximo = max(len(getDataVencimento()), len(getValorNominal()), len(getValorRecebido()))

dataaquisicao = getDataAqui()
valoraqui = getValorAqui()
datavencimento = getDataVencimento()
valornominal = getValorNominal()
valorrecebido = getValorRecebido()
nome = getNome()
produto = getP()

dataaquisicao.extend([None] * (tamanho_maximo - len(dataaquisicao)))
valoraqui.extend([None] * (tamanho_maximo - len(valoraqui)))
datavencimento.extend([None] * (tamanho_maximo - len(datavencimento)))
valornominal.extend([None] * (tamanho_maximo - len(valornominal)))
valorrecebido.extend([None] * (tamanho_maximo - len(valorrecebido)))
nome.extend([None] * (tamanho_maximo - len(nome)))
produto.extend([None] * (tamanho_maximo - len(produto)))

In [ ]:
colunas_para_preencher = {
  'Data da Aquisição': dataaquisicao,
  'Valor de Aquisição': valoraqui,
  'Data de Vencimento': datavencimento,
  'Valor Nominal': valornominal,
  'Data do Pagamento Recebido': None,
  'Valor do Pagamento Recebido': valorrecebido,
  'Tipo de Ativo': None,
  'Cedente': None,
  'Sacado': nome,
  'Produto': produto,
}

novos_dados_df = pd.DataFrame(colunas_para_preencher)

df = pd.concat([df, novos_dados_df], ignore_index=True)

df.to_excel('novatabela.xlsx', index=False)



df


In [ ]:
colunas_para_preencher = {
  'Data da Aquisição': dataaquisicao,
  'Valor de Aquisição': valoraqui,
  'Data de Vencimento': datavencimento,
  'Valor Nominal': valornominal,
  'Data do Pagamento Recebido': None,
  'Valor do Pagamento Recebido': valorrecebido,
  'Tipo de Ativo': None,
  'Cedente': None,
  'Sacado': nome,
  'Produto': produto,
}

for file1 in files1:
    file_path1= os.path.join(directory1, file1)
    data1 = read_file(file_path1)

    file2 = searchFile()
    if file2 != '':    
      file_path2= os.path.join(directory2, file2)
      data2 = read_file(file_path2)
    else:
       continue
    
    for coluna, valor in colunas_para_preencher.items():
        df[coluna] = valor

df.to_excel('novatabela.xlsx', index=False)

df


# LIMPAR PLANILHA

In [281]:
df = pd.read_excel('novatabela.xlsx')

def limpar_dados_manter_titulos(arquivo_excel, colunas):
    df_titulos = pd.DataFrame(columns=colunas)
    df_titulos.to_excel(arquivo_excel, index=False)

titulos_colunas = ['Data da Aquisição',
                   'Valor de Aquisição',
                   'Data de Vencimento',
                   'Valor Nominal',
                   'Data do Pagamento Recebido',
                   'Valor do Pagamento Recebido',
                   'Tipo de Ativo',
                   'Cedente',
                   'Sacado',
                   'Produto']

caminho_arquivo = 'novatabela.xlsx'

limpar_dados_manter_titulos(caminho_arquivo, titulos_colunas)

df


,Data da Aquisição,Valor de Aquisição,Data de Vencimento,Valor Nominal,Data do Pagamento Recebido,Valor do Pagamento Recebido,Tipo de Ativo,Cedente,Sacado,Produto
